In [7]:
import pydoc, utils_generation, os, requests, json, io
from contextlib import redirect_stdout
from utils_generation import *
from dotenv import load_dotenv
load_dotenv(override=True)


# If you do not have an LLM, just execute the prompt in chatGPT or anoter LLM free to use.
def execute_text_prompt(prompt, API_KEY = os.getenv("AZURE_OPENAI_API_KEY"), ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT"), is_json_response = None, pdf_binary = None):
    headers = {"Content-Type": "application/json","api-key": API_KEY}
    payload = {"messages" : [{ "role": "user","content": [{"type": "text","text": prompt}]}]}
    response = "DUMMY"
    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response_short = response.json()["choices"][0]["message"]["content"]
        if is_json_response:
            if response_short.startswith("```"):
                response_short = "\n".join(response_short.splitlines()[1:-1])
            response_short = json.loads(response_short, strict=False)
    except requests.RequestException as e:
        print(f"ERROR: {e}")
        return response

    return response_short

# Creates a documentation for your functions
fn_docs = "documentation.txt"
with open(fn_docs, "w") as file: file.write(pydoc.plain(pydoc.render_doc(utils_generation)))

with open(fn_docs, "r") as f: available_functions = f.read()

prompt = "What time is it?"

prompt_adapted = f"""Initial prompt is

'{prompt}'

To answer this prompt, generate a function based on available functions:

'{available_functions}'

Whenever possible, use one of the provided functions. Do not create functions on your own.

Format your response as JSON in this format:

{{"code": "xxx"}}

Do not make any imports in the code. I will handle this.

Do not put anything else in the response except the JSON.

Do not show api keys. Use os.getenv.

Print the result at the end!"""

code = execute_text_prompt(prompt_adapted, is_json_response=True)["code"]

output = io.StringIO()
with redirect_stdout(output): exec(code)
captured_output = output.getvalue()

print(code)

print(captured_output)

time = get_current_time()
print(time)
20:28:51



In [3]:
# the variable in the executed code can be used as normal variable. But as code is generated, this name always changes. Just adapt below code based on your result.

print(time_now)

20:26:02


In [6]:
def let_the_agent_work(prompt):
    prompt_adapted = f"""Initial prompt is

'{prompt}'

To answer this prompt, generate a function based on available functions:

'{available_functions}'

Whenever possible, use one of the provided functions. Do not create functions on your own.

Format your response as JSON in this format:

{{"code": "xxx"}}

Do not make any imports in the code. I will handle this.

Do not put anything else in the response except the JSON.

Do not show api keys. Use os.getenv.

Print the result at the end!"""

    code = execute_text_prompt(prompt_adapted, is_json_response=True)["code"]
    code = f"from utils_generation import *\n{code}"

    print(code)

    output = io.StringIO()
    with redirect_stdout(output): exec(code)
    captured_output = output.getvalue()

    return {"code": code, "captured_output": captured_output}

prompt = "What weather is it at my location?"
r = let_the_agent_work(prompt)

for k in r:
    print(f"{k}:\n{r[k]}")

from utils_generation import *
import os

def main():
    location = get_location_by_ip()
    city = location.get('city')
    country = location.get('country')
    if city and country:
        api_key = os.getenv('WEATHER_API_KEY')
        weather = get_weather(city, country, api_key)
        print(weather)
    else:
        print('Could not determine location')

if __name__ == '__main__':
    main()
code:
from utils_generation import *
import os

def main():
    location = get_location_by_ip()
    city = location.get('city')
    country = location.get('country')
    if city and country:
        api_key = os.getenv('WEATHER_API_KEY')
        weather = get_weather(city, country, api_key)
        print(weather)
    else:
        print('Could not determine location')

if __name__ == '__main__':
    main()
captured_output:
{'city': 'Koblenz', 'temperature_celsius': 8.3, 'humidity': 92, 'description': 'Overcast', 'wind_kph': 4.0}

